<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#include <windows.h>
#include <vector>
#include <string>
#include <iostream>
#include <cuda_runtime.h>

// NVIDIA GPUかどうかを判定する関数
bool IsNvidiaGpu(const DISPLAY_DEVICE& adapter) {
    // デバイス名に"NVIDIA"が含まれるかチェック
    const std::wstring deviceName(adapter.DeviceString);
    return deviceName.find(L"NVIDIA") != std::wstring::npos;
}

// NVIDIA GPUに接続されたモニターの数をカウントする関数
int CountNvidiaConnectedMonitors() {
    int monitorCount = 0;
    DISPLAY_DEVICE adapter = { 0 };
    adapter.cb = sizeof(DISPLAY_DEVICE);

    // すべてのディスプレイアダプターを列挙
    for (int adapterIndex = 0;
         EnumDisplayDevicesW(nullptr, adapterIndex, &adapter, 0);
         adapterIndex++) {

        if (!IsNvidiaGpu(adapter)) continue;

        DISPLAY_DEVICE monitor = { 0 };
        monitor.cb = sizeof(DISPLAY_DEVICE);

        // このアダプターに接続されたモニターを列挙
        for (int monitorIndex = 0;
             EnumDisplayDevicesW(adapter.DeviceName, monitorIndex, &monitor, 0);
             monitorIndex++) {

            if (monitor.StateFlags & DISPLAY_DEVICE_ACTIVE) {
                monitorCount++;
            }
        }
    }
    return monitorCount;
}

// CUDAが利用可能かチェック
bool IsCudaAvailable() {
    int deviceCount = 0;
    cudaError_t err = cudaGetDeviceCount(&deviceCount);
    return (err == cudaSuccess && deviceCount > 0);
}

int main() {
    // CUDAの利用可能性チェック
    if (!IsCudaAvailable()) {
        std::wcout << L"CUDA対応デバイスが見つかりません" << std::endl;
        return 0;
    }

    // NVIDIA GPU接続モニター数チェック
    const int nvidiaMonitors = CountNvidiaConnectedMonitors();

    if (nvidiaMonitors == 0) {
        std::wcout << L"警告: NVIDIA GPUに接続されたモニターが検出されません\n"
                      L"パフォーマンスの問題やエラーが発生する可能性があります"
                   << std::endl;
    } else {
        std::wcout << L"正常: NVIDIA GPUに" << nvidiaMonitors
                   << L"台のモニターが接続されています" << std::endl;
    }

    return 0;
}

In [ ]:
#include <windows.h>
#include <string>

// CUDAを使用するアプリケーションのディスプレイ構成をチェックする関数
// NVIDIAビデオカードとその他のビデオカードの両方からモニタが接続されている場合、falseを返す
bool CheckMonitorConfiguration() {
    bool nvidiaAttached = false;      // NVIDIAのディスプレイが接続されているか
    bool nonNvidiaAttached = false;   // その他の（オンボードなど）ディスプレイが接続されているか

    // ディスプレイアダプタ情報取得用構造体
    DISPLAY_DEVICE ddAdapter;
    ZeroMemory(&ddAdapter, sizeof(ddAdapter));
    ddAdapter.cb = sizeof(ddAdapter);

    // 全てのディスプレイアダプタを列挙
    for (int i = 0; EnumDisplayDevices(NULL, i, &ddAdapter, 0); i++) {
        // 各アダプタに接続されているモニタを列挙するための構造体
        DISPLAY_DEVICE ddMonitor;
        ZeroMemory(&ddMonitor, sizeof(ddMonitor));
        ddMonitor.cb = sizeof(ddMonitor);

        // 現在のアダプタに関連付けられたモニタを列挙
        for (int j = 0; EnumDisplayDevices(ddAdapter.DeviceName, j, &ddMonitor, 0); j++) {
            // モニタがデスクトップに接続されているかを確認
            if (ddMonitor.StateFlags & DISPLAY_DEVICE_ACTIVE) {
                std::wstring adapterName = ddAdapter.DeviceString;
                // アダプタ名に「NVIDIA」が含まれているかをチェック
                if (adapterName.find(L"NVIDIA") != std::wstring::npos) {
                    nvidiaAttached = true;
                } else {
                    nonNvidiaAttached = true;
                }
            }
            ZeroMemory(&ddMonitor, sizeof(ddMonitor));
            ddMonitor.cb = sizeof(ddMonitor);
        }
        ZeroMemory(&ddAdapter, sizeof(ddAdapter));
        ddAdapter.cb = sizeof(ddAdapter);
    }

    // 両方の種類のビデオカードからモニタが接続されている場合は、構成エラーとする
    if (nvidiaAttached && nonNvidiaAttached) {
        return false;
    }
    return true;
}

// エラー時にユーザーへメッセージを表示する関数
void ShowConfigurationErrorMessage() {
    MessageBox(NULL,
               L"CUDAアプリケーションは、NVIDIAビデオカードのみのモニタ接続で実行してください。",
               L"構成エラー",
               MB_OK | MB_ICONERROR);
}

int main() {
    // ディスプレイ構成をチェックする
    if (!CheckMonitorConfiguration()) {
        // 構成エラーの場合、エラーメッセージを表示して終了する
        ShowConfigurationErrorMessage();
        return -1;
    }

    // 通常のアプリケーション処理をここに記述する
    // ...

    return 0;
}

In [ ]:
#include <iostream>
#include <vector>
#include <Windows.h>
#include <cuda_runtime.h>

// Nvidia ビデオカード名の一部 (正確な名前はデバイスマネージャーで確認)
const std::string NVIDIA_CARD_NAME = "NVIDIA";

// モニターの接続状態とビデオカードを確認する関数
bool checkCudaEnvironment() {
    // 1. モニターの接続状態を確認
    DISPLAY_DEVICE displayDevice;
    displayDevice.cb = sizeof(DISPLAY_DEVICE);

    std::vector<std::string> connectedMonitorNames;
    for (DWORD i = 0; EnumDisplayDevices(NULL, i, &displayDevice, 0); ++i) {
        if (displayDevice.StateFlags & DISPLAY_DEVICE_ACTIVE) {
            connectedMonitorNames.push_back(displayDevice.DeviceName);
        }
    }

    if (connectedMonitorNames.empty()) {
        std::cerr << "エラー: 接続されたモニターがありません。" << std::endl;
        return false;
    }

    // 2. CUDA を使用可能なビデオカードを確認
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);

    if (deviceCount == 0) {
        std::cerr << "エラー: CUDA を使用可能なビデオカードがありません。" << std::endl;
        return false;
    }

    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, 0); // 最初の CUDA デバイス情報を取得

    std::string deviceName = deviceProp.name;
    if (deviceName.find(NVIDIA_CARD_NAME) == std::string::npos) {
        std::cerr << "エラー: CUDA デバイスが Nvidia ビデオカードではありません。" << std::endl;
        return false;
    }

    // 3. Nvidia ビデオカードに接続されたモニターを確認 (オプション)
    // この部分は Windows API を使用してビデオカードと接続されたモニターを直接的に確認することが難しいため、
    // 必要に応じて追加のロジックを実装する必要があります。
    // 例えば、接続されたモニターの解像度、リフレッシュレートなどを Nvidia ビデオカード情報と比較して判断できます。

    return true; // すべての条件を満たす場合は true を返します。
}

int main() {
    if (checkCudaEnvironment()) {
        std::cout << "CUDA 環境の確認完了: Nvidia ビデオカードに接続されたモニターで実行中です。" << std::endl;
        // CUDA アプリの実行
        // ...
    } else {
        std::cerr << "エラー: CUDA 環境が正しく設定されていません。" << std::endl;
        // ユーザーにメッセージを表示
        MessageBox(NULL, L"CUDA 環境が正しく設定されていません。Nvidia ビデオカードに接続されたモニターを使用してください。", L"エラー", MB_OK | MB_ICONERROR);
    }

    return 0;
}